In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Preprocess data
(only needs to be done once)

In [ ]:
from pre import preprocess
# stores preprocessed data file
preprocessed_filename = './data/preprocessed.csv'
preprocess(i='./data/training.1600000.processed.noemoticon.csv', o=preprocessed_filename, slice=1000)

### Load preprocessed data

In [ ]:
from parse import load_data

data = load_data(preprocessed_filename)
# TODO: train/dev/test split

In [ ]:
### Embedding

In [ ]:
from keras.preprocessing.text import Tokenizer
from emb import get_keras_embeddings_layer

MAX_WORDS = 5000
glove_filename = './data/glove.6B.50d.txt'
tokenizer = Tokenizer(lower=True, split=' ', oov_token="UNK")
emb_layer = get_keras_embeddings_layer(glove_filename, preprocessed_filename, tokenizer)

### Model

In [ ]:
%load_ext autoreload
%autoreload 2
from TextClassifierModel import build_model_keras, KerasTextClassifier
import pickle


# TODO: check correct arg-assignment for tokenizer
model = build_model_keras(tokenizer, emb_layer)

### Save untrained model | load it

### Train

In [ ]:
%autoreload 2
def mapping(n):
            if n == 0:
                return 0
            if n == 4:
                return 1


data_0 = data[data['target']==0]
data_4 = data[data['target']==4]

count_0, count_4 = data.target.value_counts()
print(count_0, count_4)



split = len(data['tweet'])//2

training_data = np.array(data['tweet'][:split])
training_target = np.array(data['target'][:split])
test_data = np.array(data['tweet'][split:])
test_target = np.array(data['target'][split:])

for i in range(len(training_target)):
    training_target[i] = mapping(training_target[i])
for i in range(len(test_target)):
    test_target[i] = mapping(test_target[i])
training_target = tf.keras.utils.to_categorical(training_target, 2)
test_target = tf.keras.utils.to_categorical(test_target, 2)

#print(training_target, test_target)
#doc = np.array(data['tweet'][:1])



In [ ]:
%load_ext autoreload
%autoreload 2

history = model.fit(training_data, training_target, epochs=40, batch_size=30)
# TODO: save trained model

### Evaluate dev

### Save model

In [ ]:
from TextClassifierModel import save_model_keras, load_model_keras

print('Saving')
save_model_keras(model, '/tmp/model')
print('Loading')
model = load_model_keras('/tmp/model')
print(model.model.summary())

### Evaluate production (forbidden during tuning)

### Explain prediction

In [ ]:
import eli5
from eli5.lime import TextExplainer

In [ ]:



te = TextExplainer(random_state=42)
te.fit("I love candy. I like to be positive, be happy! What a lovely day", model.predict_proba)
te.show_prediction()